# Operaciones DataFrame

## Acerca de los datos
En este cuaderno trabajaremos con 2 conjuntos de datos:
- La cotización de las acciones de Facebook a lo largo de 2018 (obtenida mediante la [`stock_analysis` package](https://github.com/stefmolin/stock-analysis)).
- Datos meteorológicos diarios de Nueva York[National Centers for Environmental Information (NCEI) API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2).

*Nota: El NCEI forma parte de la Administración Nacional Oceánica y Atmosférica (NOAA) y, como puede ver en la URL de la API, este recurso se creó cuando el NCEI se llamaba NCDC. Si la URL de este recurso cambiara en el futuro, puede buscar "NCEI weather API" para encontrar la actualizada.*

## Antecedentes de los datos meteorológicos

Significado de los datos:
- `AWND`: velocidad media del viento
- `PRCP`: precipitación en milímetros
- `SNOW`: nevadas en milímetros
- SNWD`: profundidad de la nieve en milímetros
- TMAX`: temperatura máxima diaria en grados Celsius
- TMIN`: temperatura mínima diaria en grados Celsius

## Setup

In [1]:
import numpy as np
import pandas as pd

weather = pd.read_csv('data/nyc_weather_2018.csv', parse_dates=['date'])
weather.head()

,date,datatype,station,attributes,value
0,2018-01-01,PRCP,GHCND:US1CTFR0039,",,N,0800",0.0
1,2018-01-01,PRCP,GHCND:US1NJBG0015,",,N,1050",0.0
2,2018-01-01,SNOW,GHCND:US1NJBG0015,",,N,1050",0.0
3,2018-01-01,PRCP,GHCND:US1NJBG0017,",,N,0920",0.0
4,2018-01-01,SNOW,GHCND:US1NJBG0017,",,N,0920",0.0


In [2]:
fb = pd.read_csv('data/fb_2018.csv', index_col='date', parse_dates=True)
fb.head()

,open,high,low,close,volume
date,,,,,
2018-01-02,177.68,181.58,177.5500,181.42,18151903
2018-01-03,181.88,184.78,181.3300,184.67,16886563
2018-01-04,184.90,186.21,184.0996,184.33,13880896
2018-01-05,185.59,186.90,184.9300,186.85,13574535
2018-01-08,187.20,188.90,186.3300,188.28,17994726


## Aritmética y estadística
Ya vimos que podemos utilizar operadores matemáticos como `+` y `/` con dataframes directamente. Sin embargo, también podemos utilizar métodos, que nos permiten especificar el eje sobre el que realizar el cálculo. Por defecto, esto es por columna. Busquemos las puntuaciones Z para el volumen negociado y veamos los días en los que éste se alejó más de 3 desviaciones estándar de la media:

In [3]:
fb.assign(
    abs_z_score_volume=lambda x: \
        x.volume.sub(x.volume.mean()).div(x.volume.std()).abs()
).query('abs_z_score_volume > 3')

,open,high,low,close,volume,abs_z_score_volume
date,,,,,,
2018-03-19,177.01,177.17,170.06,172.56,88140060,3.145078
2018-03-20,167.47,170.20,161.95,168.15,129851768,5.315169
2018-03-21,164.80,173.40,163.30,169.39,106598834,4.105413
2018-03-26,160.82,161.10,149.02,160.06,126116634,5.120845
2018-07-26,174.89,180.13,173.75,176.26,169803668,7.393705


Podemos utilizar `rank()` y `pct_change()` para ver qué días tuvieron el mayor cambio en el volumen negociado desde el día anterior:

In [4]:
fb.assign(
    volume_pct_change=fb.volume.pct_change(),
    pct_change_rank=lambda x: \
        x.volume_pct_change.abs().rank(ascending=False)
).nsmallest(5, 'pct_change_rank')

,open,high,low,close,volume,volume_pct_change,pct_change_rank
date,,,,,,,
2018-01-12,178.06,181.48,177.40,179.37,77551299,7.087876,1.0
2018-03-19,177.01,177.17,170.06,172.56,88140060,2.611789,2.0
2018-07-26,174.89,180.13,173.75,176.26,169803668,1.628841,3.0
2018-09-21,166.64,167.25,162.81,162.93,45994800,1.428956,4.0
2018-03-26,160.82,161.10,149.02,160.06,126116634,1.352496,5.0


El 12 de enero se conoció la noticia de que Facebook cambiaba su feed de noticias para centrarse más en los contenidos de los amigos de los usuarios que en las marcas que siguen. Dado que la publicidad de Facebook es un componente clave de su negocio ([nearly 89% in 2017](https://www.investopedia.com/ask/answers/120114/how-does-facebook-fb-make-money.asp)) se vendieron muchas acciones y la cotización cayó presa del pánico:

In [5]:
fb['2018-01-11':'2018-01-12']

,open,high,low,close,volume
date,,,,,
2018-01-11,188.40,188.40,187.38,187.77,9588587
2018-01-12,178.06,181.48,177.40,179.37,77551299


A lo largo de 2018, el precio de las acciones de Facebook nunca tuvo un mínimo por encima de 215 dólares:

In [6]:
(fb > 215).any()

open       True
high       True
low       False
close      True
volume     True
dtype: bool

Todos los precios OHLC (apertura, máximo, mínimo y cierre) de Facebook estuvieron al menos un día a 215 dólares o menos:

In [7]:
(fb > 215).all()

open      False
high      False
low       False
close     False
volume     True
dtype: bool

## Binning
Cuando trabajamos con volumen negociado, puede interesarnos más el rango de volumen que los valores exactos. No hay dos días con el mismo volumen negociado:

In [8]:
(fb.volume.value_counts() > 1).sum()

0

Podemos utilizar `pd.cut()` para crear 3 bins de rango par en volumen negociado y nombrarlos. A continuación, podemos trabajar con categorías de volumen negociado bajo, medio y alto:

In [9]:
volume_binned = pd.cut(fb.volume, bins=3, labels=['low', 'med', 'high'])
volume_binned.value_counts()

volume
low     240
med       8
high      3
Name: count, dtype: int64

Veamos los días con mayor volumen de negociación:

In [10]:
fb[volume_binned == 'high'].sort_values('volume', ascending=False)

,open,high,low,close,volume
date,,,,,
2018-07-26,174.89,180.13,173.75,176.26,169803668
2018-03-20,167.47,170.20,161.95,168.15,129851768
2018-03-26,160.82,161.10,149.02,160.06,126116634


El 25 de julio, Facebook anunció un crecimiento decepcionante del número de usuarios y las acciones se desplomaron a última hora:

In [11]:
fb['2018-07-25':'2018-07-26']

,open,high,low,close,volume
date,,,,,
2018-07-25,215.715,218.62,214.27,217.50,64592585
2018-07-26,174.890,180.13,173.75,176.26,169803668


El escándalo de Cambridge Analytica estalló el sábado 17 de marzo, así que nos fijamos en el lunes posterior para conocer las cifras:

In [12]:
fb['2018-03-16':'2018-03-20']

,open,high,low,close,volume
date,,,,,
2018-03-16,184.49,185.33,183.41,185.09,24403438
2018-03-19,177.01,177.17,170.06,172.56,88140060
2018-03-20,167.47,170.20,161.95,168.15,129851768


Como la mayoría de los días tienen un volumen similar, pero unos pocos son muy grandes, tenemos intervalos muy amplios. La mayoría de los datos se encuentran en la zona baja.

In [13]:
from visual-aids.visual_aids.misc_viz import low_med_high_bins_viz

low_med_high_bins_viz(
    fb, 'volume', ylabel='volume traded',
    title='Daily Volume Traded of Facebook Stock in 2018 (with bins)'
)

SyntaxError: invalid syntax (3524264218.py, line 1)

Si dividimos utilizando cuantiles, los intervalos tendrán aproximadamente el mismo número de observaciones. Para ello, utilizaremos `qcut()`. Haremos 4 cuartiles:

In [14]:
volume_qbinned = pd.qcut(fb.volume, q=4, labels=['q1', 'q2', 'q3', 'q4'])
volume_qbinned.value_counts()

volume
q1    63
q2    63
q4    63
q3    62
Name: count, dtype: int64

Fíjate en que los contenedores ya no cubren rangos del mismo tamaño:

In [15]:
from visual-aids.visual_aids.misc_viz import quartile_bins_viz

quartile_bins_viz(
    fb, 'volume', ylabel='volume traded', 
    title='Daily Volume Traded of Facebook Stock in 2018 (with quartile bins)'
)

SyntaxError: invalid syntax (3376702740.py, line 1)

## Aplicar Funciones
Podemos utilizar el método `apply()` para ejecutar la misma operación en todas las columnas (o filas) del marco de datos. En primer lugar, vamos a aislar las observaciones meteorológicas de la estación de Central Park y a pivotar los datos:

In [16]:
central_park_weather = weather\
    .query('station == "GHCND:USW00094728"')\
    .pivot(index='date', columns='datatype', values='value')

Calculemos las puntuaciones Z de las observaciones TMIN, TMAX y PRCP en Central Park en octubre de 2018:

In [17]:
oct_weather_z_scores = central_park_weather\
    .loc['2018-10', ['TMIN', 'TMAX', 'PRCP']]\
    .apply(lambda x: x.sub(x.mean()).div(x.std()))
oct_weather_z_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
datatype,,,,,,,,
TMIN,31.0,-1.790682e-16,1.0,-1.339112,-0.751019,-0.474269,1.065152,1.843511
TMAX,31.0,1.951844e-16,1.0,-1.305582,-0.870013,-0.138258,1.011643,1.604016
PRCP,31.0,4.655774e-17,1.0,-0.394438,-0.394438,-0.394438,-0.240253,3.936167


El 27 de octubre llovió mucho más que el resto de los días:

In [18]:
oct_weather_z_scores.query('PRCP > 3').PRCP

date
2018-10-27    3.936167
Name: PRCP, dtype: float64

De hecho, este día fue mucho más alto que el resto:

In [19]:
central_park_weather.loc['2018-10', 'PRCP'].describe()

count    31.000000
mean      2.941935
std       7.458542
min       0.000000
25%       0.000000
50%       0.000000
75%       1.150000
max      32.300000
Name: PRCP, dtype: float64

Cuando la función que queremos aplicar no está vectorizada, podemos
- utilizar `np.vectorize()` para vectorizarla (de forma similar a como funciona `map()`) y luego utilizarla con `apply()`.
- utilizar `applymap()` y pasarle directamente la función no vectorizada

Digamos que queremos contar los dígitos de los números enteros de los datos de Facebook; `len()` no está vectorizada, así que podemos usar `np.vectorize()` o `applymap()`:

In [20]:
fb.apply(
    lambda x: np.vectorize(lambda y: len(str(np.ceil(y))))(x)
).astype('int64').equals(
    fb.applymap(lambda x: len(str(np.ceil(x))))
)

C:\Users\feder\AppData\Local\Temp\ipykernel_138156\94961931.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  fb.applymap(lambda x: len(str(np.ceil(x))))


True

Una simple operación de suma a cada elemento de una serie crece linealmente en complejidad temporal cuando se utiliza `iteritems()`, pero se mantiene cerca de 0 cuando se utilizan operaciones vectorizadas. `iteritems()` y los métodos relacionados sólo deben utilizarse si no existe una solución vectorizada:

In [21]:
import time

import numpy as np
import pandas as pd

np.random.seed(0)

vectorized_results = {}
iteritems_results = {}

for size in [10, 100, 1000, 10000, 100000, 500000, 1000000, 5000000, 10000000]:
    # set of numbers to use
    test = pd.Series(np.random.uniform(size=size))
    
    # time the vectorized operation
    start = time.time()
    x = test + 10
    end = time.time()
    vectorized_results[size] = end - start
    
    # time the operation with `iteritems()`
    start = time.time()
    x = []
    for i, v in test.iteritems():
        x.append(v + 10)
    x = pd.Series(x)
    end = time.time()
    iteritems_results[size] = end - start

results = pd.DataFrame(
    [pd.Series(vectorized_results, name='vectorized'), pd.Series(iteritems_results, name='iteritems')]
).T    

# plotting
ax = results.plot(title='Time Complexity', color=['blue', 'red'], legend=False)

# formatting
ax.set(xlabel='item size (rows)', ylabel='time (s)')
ax.text(0.5e7, iteritems_results[0.5e7] * .9, 'iteritems()', rotation=34, color='red', fontsize=12, ha='center', va='bottom')
ax.text(0.5e7, vectorized_results[0.5e7], 'vectorized', color='blue', fontsize=12, ha='center', va='bottom')
for spine in ['top', 'right']:
    ax.spines[spine].set_visible(False)

AttributeError: 'Series' object has no attribute 'iteritems'

## Window Calculations

*Consultar el [`understanding_window_calculations.ipynb`](./understanding_window_calculations.ipynb) para visualizaciones interactivas mediante widgets que ayudan a comprender los cálculos de las ventanas.*

El método `rolling()` nos permite realizar cálculos de ventanas móviles. Simplemente especificamos el tamaño de la ventana (3 días aquí) y lo seguimos con una llamada a una función de agregación (suma aquí):

In [22]:
central_park_weather.loc['2018-10'].assign(
    rolling_PRCP=lambda x: x.PRCP.rolling('3D').sum()
)[['PRCP', 'rolling_PRCP']].head(7).T

date,2018-10-01,2018-10-02,2018-10-03,2018-10-04,2018-10-05,2018-10-06,2018-10-07
datatype,,,,,,,
PRCP,0.0,17.5,0.0,1.0,0.0,0.0,0.0
rolling_PRCP,0.0,17.5,17.5,18.5,1.0,1.0,0.0


También podemos realizar los cálculos continuos en todo el marco de datos a la vez. Esto aplicará la misma función de agregación a cada columna:

In [23]:
central_park_weather.loc['2018-10'].rolling('3D').mean().head(7).iloc[:,:6]

datatype,ADPT,ASLP,ASTP,AWBT,AWND,PRCP
date,,,,,,
2018-10-01,172.000000,10247.000000,10200.000000,189.000000,0.900000,0.000000
2018-10-02,180.500000,10221.500000,10176.000000,194.500000,0.900000,8.750000
2018-10-03,172.333333,10205.333333,10159.000000,187.000000,0.966667,5.833333
2018-10-04,176.000000,10175.000000,10128.333333,187.000000,0.800000,6.166667
2018-10-05,155.666667,10177.333333,10128.333333,170.333333,1.033333,0.333333
2018-10-06,157.333333,10194.333333,10145.333333,170.333333,0.833333,0.333333
2018-10-07,163.000000,10217.000000,10165.666667,177.666667,1.066667,0.000000


Podemos utilizar diferentes funciones de agregación por columna si utilizamos `agg()` en su lugar. Pasamos un diccionario asignando la columna a la agregación a realizar sobre ella. Aquí, unimos el resultado a los datos originales para ver qué ocurre:

In [24]:
central_park_weather['2018-10-01':'2018-10-07'].rolling('3D').agg(
    {'TMAX': 'max', 'TMIN': 'min', 'AWND': 'mean', 'PRCP': 'sum'}
).join( # unir con los datos originales para comparar
    central_park_weather[['TMAX', 'TMIN', 'AWND', 'PRCP']], 
    lsuffix='_rolling'
).sort_index(axis=1) # ordena las columnas para que los calcos rodantes estén junto a los originales

datatype,AWND,AWND_rolling,PRCP,PRCP_rolling,TMAX,TMAX_rolling,TMIN,TMIN_rolling
date,,,,,,,,
2018-10-01,0.9,0.900000,0.0,0.0,24.4,24.4,17.2,17.2
2018-10-02,0.9,0.900000,17.5,17.5,25.0,25.0,18.3,17.2
2018-10-03,1.1,0.966667,0.0,17.5,23.3,25.0,17.2,17.2
2018-10-04,0.4,0.800000,1.0,18.5,24.4,25.0,16.1,16.1
2018-10-05,1.6,1.033333,0.0,1.0,21.7,24.4,15.6,15.6
2018-10-06,0.5,0.833333,0.0,1.0,20.0,24.4,17.2,15.6
2018-10-07,1.1,1.066667,0.0,0.0,26.1,26.1,19.4,15.6


Supongamos que reindexamos los datos de las acciones de Facebook como hicimos con los datos del S&P 500 en el capítulo 3. Si utilizáramos cálculos continuos con estos datos, incluiríamos los valores cuando el mercado estuviera cerrado:

In [25]:
fb_reindexed = fb\
    .reindex(pd.date_range('2018-01-01', '2018-12-31', freq='D'))\
    .assign(
        volume=lambda x: x.volume.fillna(0),
        close=lambda x: x.close.fillna(method='ffill'),
        open=lambda x: x.open.combine_first(x.close),
        high=lambda x: x.high.combine_first(x.close),
        low=lambda x: x.low.combine_first(x.close)
    )
fb_reindexed.assign(day=lambda x: x.index.day_name()).head(10)

C:\Users\feder\AppData\Local\Temp\ipykernel_138156\2508490546.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  close=lambda x: x.close.fillna(method='ffill'),


,open,high,low,close,volume,day
2018-01-01,NaN,NaN,NaN,NaN,0.0,Monday
2018-01-02,177.68,181.58,177.5500,181.42,18151903.0,Tuesday
2018-01-03,181.88,184.78,181.3300,184.67,16886563.0,Wednesday
2018-01-04,184.90,186.21,184.0996,184.33,13880896.0,Thursday
2018-01-05,185.59,186.90,184.9300,186.85,13574535.0,Friday
2018-01-06,186.85,186.85,186.8500,186.85,0.0,Saturday
2018-01-07,186.85,186.85,186.8500,186.85,0.0,Sunday
2018-01-08,187.20,188.90,186.3300,188.28,17994726.0,Monday
2018-01-09,188.70,188.80,187.1000,187.87,12393057.0,Tuesday
2018-01-10,186.94,187.89,185.6300,187.84,10529894.0,Wednesday


A partir de la versión 1.0, `pandas` soporta la definición de ventanas personalizadas para cálculos rolling, lo que nos permite realizar cálculos rolling en los días en que el mercado estuvo abierto. Una forma es crear una nueva clase que herede de `BaseIndexer` y proporcionar la lógica para determinar los límites de la ventana en la clase `get_window_bounds()` (más información [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/computation.html#custom-window-rolling)). En nuestro caso, podemos utilizar la clase `VariableOffsetWindowIndexer`, que se introdujo en la versión 1.1, para realizar cálculos continuos sobre desfases temporales no fijos (como días laborables). Realicemos un cálculo móvil de tres días hábiles sobre los datos de acciones de Facebook reindexados y unámoslos a los datos reindexados para compararlos:

In [26]:
from pandas.api.indexers import VariableOffsetWindowIndexer

indexer = VariableOffsetWindowIndexer(
    index=fb_reindexed.index, offset=pd.offsets.BDay(3)
)
fb_reindexed.assign(window_start_day=0).rolling(indexer).agg({
    'window_start_day': lambda x: x.index.min().timestamp(),
    'open': 'mean', 'high': 'max', 'low': 'min',
    'close': 'mean', 'volume': 'sum'
}).join(
    fb_reindexed, lsuffix='_rolling'
).sort_index(axis=1).assign(
    day=lambda x: x.index.day_name(),
    window_start_day=lambda x: pd.to_datetime(x.window_start_day, unit='s')
).head(10)

,close,close_rolling,high,high_rolling,low,low_rolling,open,open_rolling,volume,volume_rolling,window_start_day,day
2018-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2018-01-01,Monday
2018-01-02,181.42,181.420000,181.58,181.58,177.5500,177.5500,177.68,177.680000,18151903.0,18151903.0,2018-01-01,Tuesday
2018-01-03,184.67,183.045000,184.78,184.78,181.3300,177.5500,181.88,179.780000,16886563.0,35038466.0,2018-01-01,Wednesday
2018-01-04,184.33,183.473333,186.21,186.21,184.0996,177.5500,184.90,181.486667,13880896.0,48919362.0,2018-01-02,Thursday
2018-01-05,186.85,185.283333,186.90,186.90,184.9300,181.3300,185.59,184.123333,13574535.0,44341994.0,2018-01-03,Friday
2018-01-06,186.85,186.010000,186.85,186.90,186.8500,184.0996,186.85,185.780000,0.0,27455431.0,2018-01-04,Saturday
2018-01-07,186.85,186.220000,186.85,186.90,186.8500,184.0996,186.85,186.047500,0.0,27455431.0,2018-01-04,Sunday
2018-01-08,188.28,186.632000,188.90,188.90,186.3300,184.0996,187.20,186.278000,17994726.0,45450157.0,2018-01-04,Monday
2018-01-09,187.87,187.340000,188.80,188.90,187.1000,184.9300,188.70,187.038000,12393057.0,43962318.0,2018-01-05,Tuesday
2018-01-10,187.84,187.538000,187.89,188.90,185.6300,185.6300,186.94,187.308000,10529894.0,40917677.0,2018-01-06,Wednesday


Los cálculos continuos `rolling()` utilizan una ventana deslizante. Los cálculos expansivos `expanding()`, sin embargo, crecen en tamaño. Son equivalentes a agregaciones acumulativas como `cumsum()`; sin embargo, podemos especificar el número mínimo de periodos necesarios para empezar a calcular (por defecto es 1), y no estamos limitados a agregaciones predefinidas. Por lo tanto, aunque no existe un método para la media acumulada, podemos calcularla utilizando `expanding()`. Calculemos la precipiación media del mes hasta la fecha:

In [27]:
central_park_weather.loc['2018-06'].assign(
    TOTAL_PRCP=lambda x: x.PRCP.cumsum(),
    AVG_PRCP=lambda x: x.PRCP.expanding().mean()
).head(10)[['PRCP', 'TOTAL_PRCP', 'AVG_PRCP']].T

date,2018-06-01,2018-06-02,2018-06-03,2018-06-04,2018-06-05,2018-06-06,2018-06-07,2018-06-08,2018-06-09,2018-06-10
datatype,,,,,,,,,,
PRCP,6.9,2.00,6.4,4.10,0.00,0.000000,0.000000,0.000,0.000000,0.30
TOTAL_PRCP,6.9,8.90,15.3,19.40,19.40,19.400000,19.400000,19.400,19.400000,19.70
AVG_PRCP,6.9,4.45,5.1,4.85,3.88,3.233333,2.771429,2.425,2.155556,1.97


También podemos usar `agg()` para especificar agregaciones por columna. Tenga en cuenta que esto también funciona con las funciones NumPy. Aquí, unimos los cálculos de expansión con los resultados originales para compararlos:

In [28]:
central_park_weather['2018-10-01':'2018-10-07'].expanding().agg(
    {'TMAX': np.max, 'TMIN': np.min, 'AWND': np.mean, 'PRCP': np.sum}
).join(
    central_park_weather[['TMAX', 'TMIN', 'AWND', 'PRCP']], 
    lsuffix='_expanding'
).sort_index(axis=1)

C:\Users\feder\AppData\Local\Temp\ipykernel_138156\1208072281.py:1: FutureWarning: The provided callable <function max at 0x00000245FE3C76A0> is currently using Expanding.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  central_park_weather['2018-10-01':'2018-10-07'].expanding().agg(
C:\Users\feder\AppData\Local\Temp\ipykernel_138156\1208072281.py:1: FutureWarning: The provided callable <function min at 0x00000245FE3C77E0> is currently using Expanding.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  central_park_weather['2018-10-01':'2018-10-07'].expanding().agg(
C:\Users\feder\AppData\Local\Temp\ipykernel_138156\1208072281.py:1: FutureWarning: The provided callable <function mean at 0x00000245FE3D80E0> is currently using Expanding.mean. In a future version of pandas, the provided callable will be used direct

datatype,AWND,AWND_expanding,PRCP,PRCP_expanding,TMAX,TMAX_expanding,TMIN,TMIN_expanding
date,,,,,,,,
2018-10-01,0.9,0.900000,0.0,0.0,24.4,24.4,17.2,17.2
2018-10-02,0.9,0.900000,17.5,17.5,25.0,25.0,18.3,17.2
2018-10-03,1.1,0.966667,0.0,17.5,23.3,25.0,17.2,17.2
2018-10-04,0.4,0.825000,1.0,18.5,24.4,25.0,16.1,16.1
2018-10-05,1.6,0.980000,0.0,18.5,21.7,25.0,15.6,15.6
2018-10-06,0.5,0.900000,0.0,18.5,20.0,25.0,17.2,15.6
2018-10-07,1.1,0.928571,0.0,18.5,26.1,26.1,19.4,15.6


Pandas proporciona el método `ewm()` para cálculos de medias móviles ponderadas exponencialmente. Como vimos en el capítulo 1, podemos utilizar la media móvil ponderada exponencialmente para suavizar los datos. Comparemos la media móvil con la media móvil ponderada exponencialmente con la temperatura máxima diaria. Nótese que `span` aquí son los periodos a utilizar:

In [29]:
central_park_weather.assign(
    AVG=lambda x: x.TMAX.rolling('30D').mean(),
    EWMA=lambda x: x.TMAX.ewm(span=30).mean()
).loc['2018-09-29':'2018-10-08', ['TMAX', 'EWMA', 'AVG']].T

date,2018-09-29,2018-09-30,2018-10-01,2018-10-02,2018-10-03,2018-10-04,2018-10-05,2018-10-06,2018-10-07,2018-10-08
datatype,,,,,,,,,,
TMAX,22.200000,21.100000,24.400000,25.000000,23.300000,24.400000,21.700000,20.000000,26.100000,23.300000
EWMA,24.422041,24.207716,24.220122,24.270436,24.207828,24.220226,24.057631,23.795848,23.944503,23.902922
AVG,24.723333,24.573333,24.533333,24.460000,24.163333,23.866667,23.533333,23.070000,23.143333,23.196667


*Consultar el [`understanding_window_calculations.ipynb`](./understanding_window_calculations.ipynb) para visualizaciones interactivas que ayudan a comprender los cálculos de las ventanas.*

## Pipes
Las tuberías son una forma de racionalizar nuestro código `pandas` y hacerlo más legible y flexible. Usando tuberías, podemos tomar una llamada anidada como

```python
f(g(h(data), 20), x=True)
```

y convertirlo en algo más legible:

```python
data.pipe(h)\
    .pipe(g, 20)\
    .pipe(f, x=True)\
```

Podemos utilizar tuberías para aplicar cualquier función que acepte nuestros datos como primer argumento y pasar cualquier argumento adicional. Esto facilita encadenar pasos independientemente de si son métodos o funciones:

Podemos pasar cualquier función que acepte la llamada de `pipe()` como primer argumento:

In [30]:
def get_info(df):
    return '%d rows, %d columns and max closing Z-score was %d' % (*df.shape, df.close.max())

get_info(fb.loc['2018-Q1'].apply(lambda x: (x - x.mean())/x.std()))\
    == fb.loc['2018-Q1'].apply(lambda x: (x - x.mean())/x.std()).pipe(get_info)

True

Por ejemplo, pasar `pd.DataFrame.rolling` a `pipe()` es equivalente a llamar a `rolling()` directamente en el dataframe, excepto que tenemos más flexibilidad para cambiar esto:

In [31]:
fb.pipe(pd.DataFrame.rolling, '20D').mean().equals(fb.rolling('20D').mean())

True

La tubería toma la función pasada y la llama con el objeto que llamó a `pipe()` como primer argumento. Los argumentos posicionales y de palabra clave se pasan hacia abajo:

In [32]:
pd.DataFrame.rolling(fb, '20D').mean().equals(fb.rolling('20D').mean())

True

Podemos utilizar una tubería para crear una función que podamos utilizar para todas nuestras necesidades de cálculo de ventanas:

In [33]:
from window_calc import window_calc
window_calc??

Signature: window_calc(df, func, agg_dict, *args, **kwargs)
Source:   
def window_calc(df, func, agg_dict, *args, **kwargs):
  """
    Ejecuta un cálculo de ventana de su elección en un objeto `DataFrame`.
    
    Parámetros:
        - df: El objeto `DataFrame` sobre el que ejecutar el cálculo.
        - func: El método de cálculo de la ventana que toma `df`
          como primer argumento.
        - agg_dict: Información a pasar a `agg()`, puede ser un
          diccionario que asigna las columnas a la función
          a usar, un nombre de cadena para la función,
          o la propia función.
        - args: Argumentos posicionales para pasar a `func`.
        - kwargs: Argumentos de palabra clave para pasar a `func`.
    
    Devuelve:
        Un nuevo objeto `DataFrame`.
    """
  return df.pipe(func, *args, **kwargs).agg(agg_dict)
File:      c:\users\feder\onedrive\escritorio\bootcamp data science\ds_pt_09_2023\data_analysis\analisis_de_datos\ch_04\window_calc.py
Type:      func

Ahora podemos utilizar la misma interfaz para realizar diversos cálculos de ventanas. Busquemos la mediana en expansión de los datos de Facebook:

In [34]:
window_calc(fb, pd.DataFrame.expanding, np.median).head()

c:\Users\feder\OneDrive\Escritorio\BOOTCAMP DATA SCIENCE\DS_PT_09_2023\Data_Analysis\Analisis_de_datos\ch_04\window_calc.py:22: FutureWarning: The provided callable <function median at 0x00000245FE51CAE0> is currently using Expanding.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  return df.pipe(func, *args, **kwargs).agg(agg_dict)


,open,high,low,close,volume
date,,,,,
2018-01-02,177.68,181.580,177.5500,181.420,18151903.0
2018-01-03,179.78,183.180,179.4400,183.045,17519233.0
2018-01-04,181.88,184.780,181.3300,184.330,16886563.0
2018-01-05,183.39,185.495,182.7148,184.500,15383729.5
2018-01-08,184.90,186.210,184.0996,184.670,16886563.0


El uso de la media móvil ponderada exponencialmente requiere que pasemos un argumento de palabra clave:

In [35]:
window_calc(fb, pd.DataFrame.ewm, 'mean', span=3).head()

,open,high,low,close,volume
date,,,,,
2018-01-02,177.680000,181.580000,177.550000,181.420000,1.815190e+07
2018-01-03,180.480000,183.713333,180.070000,183.586667,1.730834e+07
2018-01-04,183.005714,185.140000,182.372629,184.011429,1.534980e+07
2018-01-05,184.384000,186.078667,183.736560,185.525333,1.440299e+07
2018-01-08,185.837419,187.534839,185.075110,186.947097,1.625679e+07


Con los cálculos rodantes, podemos pasar un argumento posicional para el tamaño de la ventana:

In [36]:
window_calc(
    central_park_weather.loc['2018-10'], 
    pd.DataFrame.rolling, 
    {'TMAX': 'max', 'TMIN': 'min', 'AWND': 'mean', 'PRCP': 'sum'},
    '3D'
).head()

datatype,TMAX,TMIN,AWND,PRCP
date,,,,
2018-10-01,24.4,17.2,0.900000,0.0
2018-10-02,25.0,17.2,0.900000,17.5
2018-10-03,25.0,17.2,0.966667,17.5
2018-10-04,25.0,16.1,0.800000,18.5
2018-10-05,24.4,15.6,1.033333,1.0



<div style="display: flex; justify-content: space-between; margin-bottom: 10px;">
    <div style="text-align: left;">
        <a href="./1-consulta_y_merge.ipynb">
            <button>&#8592; Notebook Anterior</button>
        </a>
    </div>
    <div style="text-align: center;">
        <a href="./calculo_de_ventana.ipynb">
            <button>Calculo de Ventana</button>
        </a>
    </div>
    <div style="text-align: right;">
        <a href="./3-agregaciones.ipynb">
            <button>Proximo Notebook &#8594;</button>
        </a>
    </div>
</div>

<hr>